# Main Notebook

## Step 1: Load the runtime

In [ ]:
import os
os.environ.update({
    "APP_DS_SPLIT": "",
    "APP_DIRECTORY": "",
    "APP_EXPERIMENT_NAME": ""
})
configs = "configs/t5a_debug.jsonnet"

In [ ]:
# Taken from https://www.oreilly.com/library/view/python-cookbook/0596001673/ch04s23.html
def add_sys_path(new_path):
    import sys, os

    # Avoid adding nonexistent paths
    if not os.path.exists(new_path): return -1

    # Standardize the path. Windows is case-insensitive, so lowercase
    # for definiteness.
    new_path = os.path.abspath(new_path)
    if sys.platform == 'win32':
        new_path = new_path.lower(  )

    # Check against all currently available paths
    for x in sys.path:
        x = os.path.abspath(x)
        if sys.platform == 'win32':
            x = x.lower(  )
        if new_path in (x, x + os.sep):
            return 0
    sys.path.append(new_path)
    return 1

%cd ~
import os
add_sys_path(f"{os.getcwd()}/src")
from main import EntryPoint
ep = EntryPoint(configs, debug_mode=True)

RUNTIME = ep._exp
CONFIG = ep._config

## Simple Prediction

In [ ]:
from runtime.model_inference_shell import generate, predict_next_token

trainer = RUNTIME.get_loaded_trainer(load_best=True)
model = trainer.model

In [ ]:
generate(RUNTIME, model, "jump twice", max_length=100)

In [ ]:
predict_next_token(RUNTIME, model,
                   "3 1 4 9 3 8 4 2 4 + 5 9 4 1 6 7 5 4 2\n<scratch> 2 + 2 , C: 0#4 + 5 , 4 C: 0#8 +",
                   target_tokens=[])

## JavaScript Visualization

In [ ]:
! pip install --user -qq ecco

In [ ]:
from ecco import LM
from ecco.util import load_config

trainer = RUNTIME.get_loaded_trainer(load_best=True)
model = trainer.model

model.config.output_attentions = True
model.config.output_hidden_states = True

lm = LM(model, RUNTIME.tokenizer,
        model_name=model.config._name_or_path,
        config=load_config(model.config._name_or_path),
        collect_activations_flag=False,
        collect_activations_layer_nums=None,
        verbose=True,
        gpu=True)

In [ ]:
output = lm.generate("3 1 4 9 3 8 4 2 4 + 5 9 4 1 6 7 5 4 2\n", generate=400, do_sample=False, attribution=['ig', 'grad_x_input'])

In [ ]:
output.primary_attributions(attr_method='ig', style="detailed")

In [ ]:
output.layer_predictions(position=43, topk=5)

In [ ]:
output.rankings()